In [8]:
class User:
    def __init__(self,id,name='Kunal Sharma'):
        if not isinstance(id,int):
            raise TypeError(f'Expected id to be an integer, got {type(id).__name__}')

        if not isinstance(name,str):
            raise TypeError(f'Expected name to be a string, got {type(name).__name__}')

        self.id = id
        self.name = name

try:
    user = User(id=1,name=2)
except TypeError as e:
    print(e)



Expected name to be a string, got int


In [10]:
from pydantic import BaseModel

class User(BaseModel):
    id: int
    name: str='Dr Strange Variant'

user = User(id=1,name='Kunal Sharma')
print(user)



id=1 name='Kunal Sharma'


In [11]:
print(user.model_fields_set)

{'id', 'name'}


In [12]:
print(user.model_dump())
print(user.model_dump_json())
print(user.model_json_schema())

{'id': 1, 'name': 'Kunal Sharma'}
{"id":1,"name":"Kunal Sharma"}
{'properties': {'id': {'title': 'Id', 'type': 'integer'}, 'name': {'default': 'Dr Strange Variant', 'title': 'Name', 'type': 'string'}}, 'required': ['id'], 'title': 'User', 'type': 'object'}


In [16]:
from typing import List, Optional
from pydantic import BaseModel

class Food(BaseModel):
    name: str
    price: float
    ingredients: Optional[List[str]]=NotImplemented

class Restaurant(BaseModel):
    name: str
    location:str
    menu:List[Food]

restaurant_instance_1 = Restaurant(
    name="Three Broomsticks",
    location="Hogwarts",
    menu=[Food(name="Butterbeer",price=5.99,ingredients=["sugar","butter","beer"])]

)

restaurant_instance_2 = Restaurant(
    name="The Leaky Cauldron",
    location="Diagon Alley",
    menu=[
        {"name": "Pumpkin Juice", "price" : 3.99, "ingredients": ["pumpkin","juice"]},
        {"name": "Wizarding World Coffee", "price" : 4.99, "ingredients": ["coffee","magic"]},
        {"name": "Butterbeer", "price" : 5.99, "ingredients": ["sugar","butter","beer"]},
    ]
)

#print(restaurant_instance_1)
print(restaurant_instance_2)

name='The Leaky Cauldron' location='Diagon Alley' menu=[Food(name='Pumpkin Juice', price=3.99, ingredients=['pumpkin', 'juice']), Food(name='Wizarding World Coffee', price=4.99, ingredients=['coffee', 'magic']), Food(name='Butterbeer', price=5.99, ingredients=['sugar', 'butter', 'beer'])]


In [20]:
!pip install "pydantic[email]"


  Using cached email_validator-2.2.0-py3-none-any.whl.metadata (25 kB)
  Using cached dnspython-2.7.0-py3-none-any.whl.metadata (5.8 kB)
Using cached email_validator-2.2.0-py3-none-any.whl (33 kB)
Using cached dnspython-2.7.0-py3-none-any.whl (313 kB)


In [21]:
from pydantic import EmailStr, PositiveInt, conlist, Field, HttpUrl

class Address(BaseModel):
    street:str
    city: str
    state: str
    zip_code:int

class Employee(BaseModel):
    name: str
    email: EmailStr
    age: PositiveInt

class Owner(BaseModel):
    name: str
    email: EmailStr

class Restaurant(BaseModel):
    name: str=Field(..., pattern=r"^[a-zA-Z0-9- ]+")
    owner: Owner
    employees: conlist(Employee, min_length=2)
    number_of_seats: PositiveInt
    delivery_available: bool=False
    website: HttpUrl
    

restaurant_instance_3 = Restaurant(
    name="The Leaky Cauldron",
    owner= {"name": "Albus Dumbledore", "email": "albus@hogwarts.edu"},
    employees=[
        {"name": "Minerva McGonagall", "email": "minerva@hogwarts.edu", "age": 30},
        {"name": "Severus Snape", "email": "severus@hogwarts.edu", "age": 35},
    ],
    number_of_seats=100,
    delivery_available=True,
    website="https://hogwarts.edu"
)

print(restaurant_instance_3)



name='The Leaky Cauldron' owner=Owner(name='Albus Dumbledore', email='albus@hogwarts.edu') employees=[Employee(name='Minerva McGonagall', email='minerva@hogwarts.edu', age=30), Employee(name='Severus Snape', email='severus@hogwarts.edu', age=35)] number_of_seats=100 delivery_available=True website=Url('https://hogwarts.edu/')


In [25]:
from pydantic import field_validator

class Wizard(BaseModel):
    name: str
    age: int

    @field_validator('age')
    def validate_age(cls,v):
        if v < 0:
            raise ValueError('Age must be positive')
        return v

    @field_validator('name')
    @classmethod
    def name_must_contain_space(cls,v):
        if ' ' not in v:
            raise ValueError('Wizard name must contain a space')
        return v

try:
    wizard_instance = Wizard(name='Nicolas Flamel',age=-662)
except ValueError as e:
    print(e)



1 validation error for Wizard
age
  Value error, Age must be positive [type=value_error, input_value=-662, input_type=int]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error


In [26]:
from uuid import uuid4

class WizardId(BaseModel):
    id: int=Field(default_factory=lambda: uuid4().hex)

wizard_id_instance = WizardId()
print(wizard_id_instance)

id='5d1ef6aafa1444a2964cf566bd06cffe'


In [29]:
from typing import List
from pydantic import BaseModel, Field, EmailStr
from decimal import Decimal

class WizardRegistration(BaseModel):
    username: str =  Field(..., min_length=3, max_length = 30, pattern=r'^\w+')
    email: EmailStr = Field(...)
    age: int = Field(..., gt=0, lt=120)
    height: float= Field(..., gt=0.5, lt=2.0)
    house: str = Field(..., pattern=r'^Gryffindor|Hufflepuff|Ravenclaw|Slytherin$')
    wand_wood: str = Field(..., pattern=r'^oak|ash|vine|holly|phoenix|phoenix|phoenix|phoenix$')

wizard_registration_instance = WizardRegistration(
    username='Harry Potter',
    email='harry@hogwarts.edu',
    age=11,
    height=1.75,
    house='Gryffindor',
    wand_wood='oak'
)

print(wizard_registration_instance)

username='Harry Potter' email='harry@hogwarts.edu' age=11 height=1.75 house='Gryffindor' wand_wood='oak'


In [30]:
!pip install -qU 'pydantic-settings'

In [35]:
from pydantic_settings import BaseSettings, SettingsConfigDict
from pydantic import Field

# validation_alias: Used only during validation/parsing
    # Useful when the input name differs from the model field name
# alias: Used during serialization/deserialization
    # Useful when the output name differs from the model field name
class Settings(BaseSettings):
    auth_key: str = Field(validation_alias='my_auth_key')
    api_key: str = Field(alias='my_api_key')
    
    model_config = SettingsConfigDict(
    env_file='.env',                # Path to environment file
    env_file_encoding='utf-8',      # Encoding of the file
    case_sensitive=True,            # Whether env vars are case sensitive
    extra='ignore',                 # How to handle extra fields
)

settings = Settings()
print(settings.model_dump())

{'auth_key': 'alohomora_vault_713', 'api_key': 'goblin_approved_9_3_4'}
